In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection, svm , naive_bayes
from sklearn.naive_bayes import ComplementNB , BernoulliNB,GaussianNB,MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score,balanced_accuracy_score,average_precision_score,f1_score,recall_score,classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV
import pickle

# Linear SVM model

In [ ]:
train=pd.read_csv(r'/content/dataset.csv', encoding='utf-8')
train

In [ ]:
# Split the dataset
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(train["Tweets"],train["sentiment"],random_state=1,test_size=0.10)

Train_X

1520     الاختيار بيدك الزم بيتك كلنا مسؤول نعي لنقي و...
7331     نبيها صفر يارب يختفي هالمرض قبل رمضان  ونصلي ...
5321     جميل  الدراسه مستمره التعليم عن بعد تعليم صبي...
6725     لا يظهر الوباء في قوم الا لكثره ذنوبهم او لفج...
6657     كوريا الجنوبيه الدوله الوحيده التي ما عملت حظ...
                              ...                        
7935    مباشره الوضع حاليا في احد المحلات التجاريه في ...
5192                   تقوي اله امان لك ولذريتك حظر كامل 
3980     اليوم في تمام مساء علي المنصه الانستغرام في ا...
235              ابطال الصحه كان تكون صباره وامنيتك عناق 
5157     تفصيل حسب الطلب نبتكر ونفذ ماترغب به عليك الا...
Name: Tweets, Length: 7200, dtype: object

In [ ]:
# Feature extraction
vectorizer = CountVectorizer(max_features=20000,decode_error="ignore")
Train_X= vectorizer.fit_transform(Train_X)
Test_X = vectorizer.transform(Test_X)

Train_X

<7200x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 118002 stored elements in Compressed Sparse Row format>

In [ ]:
# Linear SVM Classifier
clf=svm.LinearSVC(C=0.1)

start = time.time()
# fit the training dataset on the classifier
clf.fit(Train_X, Train_Y)

end = time.time()
#Print the elapsed time for training the model
print("The elapsed time ", end - start)



The elapsed time  0.06572175025939941


In [ ]:
# predict the labels on test dataset
clf_predictions=clf.predict(Test_X)

# Use accuracy_score function to get the accuracy
print("f1 Score -> ",f1_score(Test_Y,clf_predictions,average='micro')*100)
print("recall Score -> ",recall_score(Test_Y,clf_predictions,average='micro')*100)
print("balanced Accuracy Score -> ",balanced_accuracy_score(Test_Y,clf_predictions)*100)

f1 Score ->  84.625
recall Score ->  84.625
balanced Accuracy Score ->  84.6137254534201


In [ ]:
filename = 'finalized_model.pkl'
pickle.dump(clf, open(filename, 'wb'))

with open(filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
#Predicting on a new sample and saving the results in csv format
sample_sub=pd.read_csv(r'/content/Clean2020-04-01.csv', encoding='utf-8')
Final = vectorizer.transform(sample_sub['Tweets'])

y_pre=Pickled_LR_Model.predict(Final)
sub=pd.DataFrame({'Tweets':sample_sub['Tweets'].values.tolist(),'sentiment':y_pre})
sub.to_csv(r'SVM.csv',encoding= 'utf-8-sig',index=False)